## CONSIGNA TP2:

## 1. Análisis y curación de datos de una API - formato JSON

Referencias: https://darksky.net/dev/docs

  1. Extraer los datos para la localidad Santa Rosa de Calamuchita para el período 1/Mayo/2018 al 01/Mayo/2020.
  2. ¿Existen valores faltantes? Cuantificarlos.
  3. Generar un nuevo dataset a partir de los datos (time, summary, precipAccumulation, temperature, humidity, pressure, windSpeed, uvIndex, temperatureMin, temperatureMax.
  4. Hacer las conversiones de unidades necesarias a estos datos para poder compararlos con "APRHi - Sta Rosa de Calamuchita".
  5. Comparar los estadísticos de estos datasets con los de "APRHi - Sta Rosa de Calamuchita".


## 2. Análisis y curación de datos Scrapeados de WeatherUnderground

Referencias: https://www.wunderground.com/

  1. Extraer los datos para la estación meteorológica personal (PWS, por siglas en inglés) "ICALAMUC5" (en Villa General Belgrano), para el período 1/Mayo/2018 al 01/Mayo/2020 mediante el Scrappeo de WeatherUnderground (WU).
  2. ¿Existen valores faltantes? Cuantificarlos.
  3. Convertir los datos faltantes en NaN.
  4. Hacer las conversiones de unidades necesarias a estos datos para poder compararlos con "APRHi - Sta Rosa de Calamuchita".
  5. Comparar los estadísticos de este dataset con los de "APRHi - Sta Rosa de Calamuchita".


## 3. Análisis y curación de datos de un Excel y un TXT

Referencias GLDAS: https://ldas.gsfc.nasa.gov/gldas
Referencias IMERG: https://disasters.nasa.gov/instruments/imerg

  1. Leer los datos de GLDAS_SantaRosa.xlsx para el período 1/Mayo/2018 al 31/Abril/2020.
  2. Leer los datos de IMERG_SantaRosa.txt para el período 1/Mayo/2018 al 31/Abril/2020.
  3. Hacer las conversiones de unidades necesarias a GLDAS para poder compararlos con "APRHi - Sta Rosa de Calamuchita".
  4. Comparar los estadísticos de estos datasets con los de "APRHi - Sta Rosa de Calamuchita".
  
  
## 4. Rellenar datos faltantes
  1. ¿De todos los datasets comparados, para cada variable cuál se parece más a los de "APRHi - Sta Rosa de Calamuchita"?
  2. Acumular las lluvias y promediar las demás variables de "APRHi - Sta Rosa de Calamuchita" cada 30 minutos.
  3. Interpolar los datos ya procesados de las mejores fuentes de información "alternativas" (salvo las lluvias), cada 30 minutos.
  4. Hacer lo mismo que en el punto anterior pero para la mejor fuente "alternativa" de lluvias, dividiendo por el número de "time-steps" equivalentes para llevar a cada dato a 30 minutos. (HINT: si la mejor es IMERG, no hacer nada ;D).
  5. Interpolar el dataset "APRHi - Sta Rosa de Calamuchita" para rellenar los huecos.
  6. Por otro lado, utilizar lo hecho en los punto 3 y 4 para rellenar los datos faltantes en la estación de "APRHi - Sta Rosa de Calamuchita".
  7. Exportar los dos datasets nuevos en formato csv.


## Ayuda Parte 1

In [2]:
import datetime
import requests
import pandas as pd
import json
from pandas import json_normalize

In [3]:
# This function returns a list of date between to boundaries. Source: https://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)

In [4]:
baseUrl = "https://api.darksky.net/forecast/<key>/<longitude>,<latitude>,<date>?exclude=flags,hourly&units=ca"

In [26]:
"""
En este bloque unicamente quise probar los metodos para pasar entre date, datetime y timestamp,
ya que son los formatos que se usan en la api.
Lo dejo aca porque tal vez le pueda ser ilustrativo a alguien
"""

start_date = datetime.datetime(2018, 5, 1)
end_date = datetime.datetime(2020, 5, 1)

for date in daterange(start_date, end_date):
        date2 = date.timestamp()
        tul = datetime.datetime.fromtimestamp(date2)
        date1 = date.strftime("%Y-%m-%d")
        print(date1, date2, tul.date())

2018-05-01 1525143600.0 2018-05-01
2018-05-02 1525230000.0 2018-05-02
2018-05-03 1525316400.0 2018-05-03
2018-05-04 1525402800.0 2018-05-04
2018-05-05 1525489200.0 2018-05-05
2018-05-06 1525575600.0 2018-05-06
2018-05-07 1525662000.0 2018-05-07
2018-05-08 1525748400.0 2018-05-08
2018-05-09 1525834800.0 2018-05-09
2018-05-10 1525921200.0 2018-05-10
2018-05-11 1526007600.0 2018-05-11
2018-05-12 1526094000.0 2018-05-12
2018-05-13 1526180400.0 2018-05-13
2018-05-14 1526266800.0 2018-05-14
2018-05-15 1526353200.0 2018-05-15
2018-05-16 1526439600.0 2018-05-16
2018-05-17 1526526000.0 2018-05-17
2018-05-18 1526612400.0 2018-05-18
2018-05-19 1526698800.0 2018-05-19
2018-05-20 1526785200.0 2018-05-20
2018-05-21 1526871600.0 2018-05-21
2018-05-22 1526958000.0 2018-05-22
2018-05-23 1527044400.0 2018-05-23
2018-05-24 1527130800.0 2018-05-24
2018-05-25 1527217200.0 2018-05-25
2018-05-26 1527303600.0 2018-05-26
2018-05-27 1527390000.0 2018-05-27
2018-05-28 1527476400.0 2018-05-28
2018-05-29 152756280

In [81]:
"""
En este bloque se descarga la data y se la escribe en un archivo.
OJO! Cada vez que se ejecuta la funcion se sobre escribe el archivo con la data scrapeada.
"""

start_date = datetime.datetime(2018, 5, 1)
end_date = datetime.datetime(2018, 5, 5)

data = []
for date in daterange(start_date, end_date):
    tmstmp = str(int(date.timestamp()))
    replacedUrl = baseUrl.replace("<key>", "9349598e9f11f5eddceb6791daa6d787").replace("<longitude>", "-64.533333").replace("<latitude>", "-32.066667").replace("<date>",tmstmp)
    r = requests.get(url = replacedUrl)
    # json with day data
    partial_data = r.json()
    data.append(partial_data)
# writing json
with open('sta_rosa_scraped.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [121]:
with open('sta_rosa_scraped_last.json') as f:
    data = json.load(f)

In [122]:
data

[{'latitude': -64.533333,
  'longitude': -32.066667,
  'timezone': 'Etc/GMT+2',
  'currently': {'time': 1525143600, 'uvIndex': 0},
  'offset': -2},
 {'latitude': -64.533333,
  'longitude': -32.066667,
  'timezone': 'Etc/GMT+2',
  'currently': {'time': 1525230000, 'uvIndex': 0},
  'offset': -2},
 {'latitude': -64.533333,
  'longitude': -32.066667,
  'timezone': 'Etc/GMT+2',
  'currently': {'time': 1525316400, 'uvIndex': 0},
  'offset': -2},
 {'latitude': -64.533333,
  'longitude': -32.066667,
  'timezone': 'Etc/GMT+2',
  'currently': {'time': 1525402800, 'uvIndex': 0},
  'offset': -2},
 {'latitude': -64.533333,
  'longitude': -32.066667,
  'timezone': 'Etc/GMT+2',
  'currently': {'time': 1525489200, 'uvIndex': 0},
  'offset': -2},
 {'latitude': -64.533333,
  'longitude': -32.066667,
  'timezone': 'Etc/GMT+2',
  'currently': {'time': 1525575600, 'uvIndex': 0},
  'offset': -2},
 {'latitude': -64.533333,
  'longitude': -32.066667,
  'timezone': 'Etc/GMT+2',
  'currently': {'time': 15256620

In [123]:
flatten_json = pd.json_normalize(data) # esta linea en teoria devuelve el json desanidado
flatten_json # pero como se ve toda la data anidada en la llave daily queda comprimida en daily.data.

,latitude,longitude,timezone,offset,currently.time,currently.uvIndex,currently.summary,currently.icon,currently.precipIntensity,currently.precipProbability,...,currently.dewPoint,currently.humidity,currently.pressure,currently.windSpeed,currently.windGust,currently.windBearing,currently.cloudCover,currently.visibility,currently.ozone,daily.data
0,-64.533333,-32.066667,Etc/GMT+2,-2,1525143600,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-64.533333,-32.066667,Etc/GMT+2,-2,1525230000,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-64.533333,-32.066667,Etc/GMT+2,-2,1525316400,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-64.533333,-32.066667,Etc/GMT+2,-2,1525402800,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-64.533333,-32.066667,Etc/GMT+2,-2,1525489200,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,-64.533333,-32.066667,Etc/GMT+2,-2,1583809200,0,Overcast,cloudy,0.0284,0.05,...,-8.45,0.57,997.3,29.08,29.08,146.0,1.00,1.417,268.7,"[{'time': 1583805600, 'summary': 'Foggy in the..."
680,-64.533333,-32.066667,Etc/GMT+2,-2,1583895600,0,Partly Cloudy,partly-cloudy-night,0.0000,0.00,...,-5.58,0.66,1001.5,22.75,26.41,197.0,0.57,16.093,283.0,"[{'time': 1583892000, 'summary': 'Windy starti..."
681,-64.533333,-32.066667,Etc/GMT+2,-2,1583982000,0,Windy and Mostly Cloudy,wind,0.0000,0.00,...,-4.44,0.69,1004.2,40.72,49.60,229.0,0.81,16.093,272.8,"[{'time': 1583978400, 'summary': 'Windy and pa..."
682,-64.533333,-32.066667,Etc/GMT+2,-2,1584068400,0,Windy and Overcast,wind,0.0246,0.01,...,-4.16,0.72,1006.3,47.21,56.98,240.0,0.95,16.093,264.9,"[{'time': 1584064800, 'summary': 'Windy and mo..."


In [124]:
# identificar a partir de que fecha hay informacion, y luego descartar la informacion vacia

for n,dat in enumerate(data):
    try:
        if data[n]['daily']:
            print(n)
        break
    except KeyError:
        continue

219


In [125]:
cleaned_json = data[219::]
flatten_json = pd.json_normalize(cleaned_json, ['daily','data'])
flatten_json.columns

Index(['time', 'summary', 'icon', 'sunriseTime', 'sunsetTime', 'moonPhase',
       'precipIntensity', 'precipIntensityMax', 'precipIntensityMaxTime',
       'precipProbability', 'precipType', 'precipAccumulation',
       'temperatureHigh', 'temperatureHighTime', 'temperatureLow',
       'temperatureLowTime', 'apparentTemperatureHigh',
       'apparentTemperatureHighTime', 'apparentTemperatureLow',
       'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure',
       'windSpeed', 'windGust', 'windGustTime', 'windBearing', 'cloudCover',
       'uvIndex', 'uvIndexTime', 'visibility', 'ozone', 'temperatureMin',
       'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime',
       'apparentTemperatureMin', 'apparentTemperatureMinTime',
       'apparentTemperatureMax', 'apparentTemperatureMaxTime'],
      dtype='object')

In [126]:
df = flatten_json[['time',
                   'summary',
                   'precipAccumulation',
                   'humidity',
                   'pressure',
                   'windSpeed',
                   'uvIndex',
                   'temperatureMin',
                   'temperatureMax']]

In [153]:
df.set_index('time').head()

,summary,precipAccumulation,humidity,pressure,windSpeed,uvIndex,temperatureMin,temperatureMax
time,,,,,,,,
1544061600,Foggy overnight.,0.6,0.72,982.8,11.30,4,-3.17,-0.11
1544148000,Snow (3–7 cm.) until evening.,4.4,0.78,959.3,37.46,3,-3.72,1.70
1544234400,Foggy in the morning.,1.4,0.78,951.3,32.02,3,-0.64,1.01
1544320800,Mostly cloudy throughout the day.,0.3,0.67,966.8,25.85,4,-2.67,0.11
1544407200,Foggy in the morning.,0.8,0.68,975.9,29.48,3,-3.18,0.04


In [167]:
df2 = df

KeyError: '2020-04-30'

In [166]:
first_day_data = df['time'][0]
date = datetime.datetime.fromtimestamp(first_day_data)
date.date().strftime("%Y-%m-%d")

'2018-12-05'

## Ayuda Parte 2

In [ ]:
#!pip3 install requests bs4

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from urllib.request import urlopen
import datetime
import pandas as pd

In [ ]:
stations = ["ICALAMUC5"]
start_date = datetime.date(2018, 9, 1)
end_date = datetime.date(2018, 9, 10) #Esta fecha no es incluida
rango = 'daily'

baseUrl = 'https://www.wunderground.com/dashboard/pws/<station>/table/<date>/<date>/daily'

In [ ]:
tab_exist = 0
#stations_temp = []
  
for station in stations:
  print(station)
  for date in daterange(start_date, end_date):
    url = baseUrl.replace("<date>", date.strftime("%Y-%m-%d")).replace("<station>", station)
    page = bs(requests.get(url).content)
    response = requests.get(url)
    #if response != 200:
    #print(response)
    tabla = page.find('table', attrs={'class':'history-table desktop-table'})
    
    if tab_exist == 0:                

      
      print(response)
      print(date)
      #exist = 0
      
      if tabla:
        print('Hay datos', date, station)
        tab_exist = 1
        #Creo lista con el nombre de las columnas
        list_html = tabla.find_all('th')
        #lista con las unidades
        list_unid = ['','C','C','%','','km/h','km/h','hPa','mm','mm','','']
        list_col = []
        for i in list_html:
          col_temp = i.string
          list_col.append(col_temp)
        for j in range(len(list_col)):
          if list_unid[j]:          
            list_col[j] = list_col[j] + '[' + list_unid[j] + ']'
            #list_col[j] = list_col[j].strip('()')

        df_wu = pd.DataFrame(columns = list_col )
  
  
    if tab_exist == 1:    #Entra acá cuando ya se creo la lista con los parametros
      print(date)
      #Entra acá cuando ya se creo la lista
      if response.status_code == 200:
        if tabla:
          m = tabla.tbody.find_all('tr')
          m_val = []
          for i in m:
            list_temp = []
            l_html = i.find_all('td')
            
            for j in range(0,len(l_html)): 
              if j == 0:
                
                list_temp.append(date.strftime("%Y-%m-%d") +' '+ l_html[j].string)            
                        
              elif j == 4 or j == 10 or j == 11: 
                list_temp.append(l_html[j].string) 
              else:
                list_temp.append(l_html[j].span.span.string)

            m_val.append(list_temp) #Matriz con los valores

          df_temp = pd.DataFrame(m_val, columns=list_col)  #Transformo la matriz en un DataFrame
      #df_temp.rename(columns={0:'Time'})
          #for i in range(0,len(m_val[0])):
            #df_temp.rename(columns={i:list_col[i]}, inplace=True)
          df_temp['Station'] = station
          df_temp['Date'] = date

          df_wu =pd.concat([df_wu,df_temp], axis=0)
      else:
        print(response)
        

In [ ]:
df_wu